In [ ]:
##########################################################
# 1. split train data into training set and test set
# 2. train Q-Tables on Training set
# 3. make suggestions for test set
# 4. Calculate Metrics 1 for our suggestions
#########################################################

In [3]:
# 1. Split Train 
from DataPrep import *
userFeaturesTrain, recItemsTrain, purchaseLabelTrain, userFeaturesTest, recItemsTest, purchaseLabelTest = splitTrainSet()

Number of Multiprocessing threads: 31


In [5]:
# 2.
import numpy as np
from tqdm import tqdm
import os
os.chdir('/tf/shared')
from classes.QLearning import *
N_ITEMS = 381

# predict user cluster label for users in the training set
NCLUSTERS = 50
ClusterModel, clusterLabels = getClusterLabels()
clusterLabelTrain  = ClusterModel.predict(userFeaturesTrain)
# load item price
itemInfo = pd.read_csv('/tf/shared/item_info.csv', ' ')
itemInfo = itemInfo.sort_values(by = 'item_id')
itemPrice = itemInfo.price
# train Q tables 
QLModels = []
for i in range(NCLUSTERS):
    QLModels.append(QLearning(n_states = N_ITEMS+1, n_actions = N_ITEMS+1))

# to train Q tables: 
#### state: the last item recommended to users, 0 if game just started, otherwise = itemID
#### action: the next item to recommend
#### reward: (item is purchased) * price
#### to feed a set of (state, action, reward) to a Q table
for i in tqdm(range(len(recItemsTrain))):
# loop thru samples
    recItems = recItemsTrain[i]
    purLabel = purchaseLabelTrain[i]
    for j, item in enumerate(recItems):
        if j>2 and purLabel[0]*purLabel[1]*purLabel[2]==0:
            # don't train if game stopped
            break
        if j>5 and purLabel[3]*purLabel[4]*purLabel[5]==0:
            # don't train if game stopped
            break
        # calculate state:
        if j==0:
            state = 0
        else:
            state = recItems[j-1]
        # calculate action:
        action = item
        # calculate rewards, note: itemPrice is an array, itemID from raw data is 1-based index
        reward = itemPrice[item-1] * purLabel[j]
        train_data = np.array([(state, action, reward)])
        # predict user cluster label of this sample based on user features
        clusterID = clusterLabelTrain[i]
        QLModels[clusterID].train(train_data)


100%|██████████| 208069/208069 [01:24<00:00, 2465.20it/s]


In [6]:
# 3. make suggestion for test set
# predict user cluster label for users in the test set
clusterLabelTest  = ClusterModel.predict(userFeaturesTest)

# make suggestion for each test sample
items_out = []
for cluster in tqdm(clusterLabelTest):
# clusterLabelTest is array of labels of users in test set
    model = QLModels[cluster]
    items_out.append(model.nextNSteps(0, 9))

100%|██████████| 52018/52018 [00:46<00:00, 1109.56it/s]


In [11]:
# 4. calculate metrics
from classes.Metrics import *
metrics = Metrics(recItemsTest, purchaseLabelTest)
score = metrics.calculate_metrics2(items_out, w3 = 1000)
print(score) # max score possible
# calculate metrics of test set (max score possible by ground truth)
score_max = metrics.calculate_metrics2(recItemsTest, w3 = 1000)
print(score_max) # max score possible
print('percentage of max score: ' + str(score/score_max))

65694232
158433981612
percentage of max score: 0.00041464735867639287
